# **Examen 2024-2025**

In [1]:
import sys
import os
from pathlib import Path
current_dir = Path(os.getcwd())
parent_dir = current_dir.parent
sys.path.append(str(parent_dir))

from algorithm import AES, CESAR, DIFFIE_HELLMANN, ECC, ELGAMAL, RSA, VIGENERE
from compute import BASIC_COMPUTE, LFSR

## **Exercice 1**

1. Chiffrement de César avec k = 3

In [2]:
my_cesar = CESAR(k=3)
_ = my_cesar.encrypt(m="E.T telephone maison")

Message en clair : E.T telephone maison
Message chiffré avec pour cle 3 (valeur du décalage) : h.w whohskrqh pdlvrq


2. Chiffrement de Vigenère avec k="BRAVE"

In [3]:
my_vigenere = VIGENERE(k="BRAVE")
_ = my_vigenere.encrypt(m="E.T telephone maison")

Message en clair : E.T telephone maison
Message chiffré avec pour cle "brave" : f.k tzpfghjrf dadwpe


## **Exercice 2**

Au cours du calcul d’un chiffrement AES, la valeur du tableau à l’entrée du tour est :  

e0 c8 d9 85  
92 63 b1 b8  
7f 63 35 be  
e8 c0 50 01  

**1. Calculer la valeur du tableau après SubBytes...**

In [4]:
import ctypes

state = (ctypes.c_uint8 * 16)(
    0xe0, 0xc8, 0xd9, 0x85, 
    0x92, 0x63, 0xb1, 0xb8, 
    0x7f, 0x63, 0x35, 0xbe, 
    0xe8, 0xc0, 0x50, 0x01
)

my_aes = AES()
my_aes.sub_bytes(state)

State after SubBytes (4x4):
e1 e8 35 97 
4f fb c8 6c 
d2 fb 96 ae 
9b ba 53 7c 



**2. Puis après ShiftRows...**

In [5]:
state_after_sub_bytes = (ctypes.c_uint8 * 16)(
    0xe1, 0xe8, 0x35, 0x97, 
    0x4f, 0xfb, 0xc8, 0x6c, 
    0xd2, 0xfb, 0x96, 0xae, 
    0x9b, 0xba, 0x53, 0x7c
)

my_aes.shift_rows(state_after_sub_bytes)

State after ShiftRows (4x4):
e1 e8 35 97 
fb c8 6c 4f 
96 ae d2 fb 
7c 9b ba 53 



**3. Détails des calculs sur feuille**  

**4. Puis après MixColumns.**

In [6]:
state_after_shift_rows = (ctypes.c_uint8 * 16)(
    0xe1, 0xe8, 0x35, 0x97, 
    0xfb, 0xc8, 0x6c, 0x4f, 
    0x96, 0xae, 0xd2, 0xfb, 
    0x7c, 0x9b, 0xba, 0x53
)

my_aes.mix_columns(state_after_shift_rows)

State after MixColumns (4x4):
25 bd b6 4c 
d1 11 3a 4c 
a9 d1 33 c0 
ad 68 8e b0 



## **Exercice 3**

On considère un chiffrement RSA avec le module n = 493, et l’exposant public e = 33.  

**1. Calculer le chiffré c1 du message clair m1 = 110.**

In [7]:
my_rsa = RSA(n=1537, e=17)
c1 = my_rsa.encrypt(m=681)


c = m^e mod n

Square and Multiply: 681 ^ 17 mod 1537
	bin(17) = 10001
	0: Square: (1 * 1) % 1537 = 1
	   Multiply: (1 * 681) % 1537 = 681
	1: Square: (681 * 681) % 1537 = 1124
	   Pas de multiplication bit = 0
	2: Square: (1124 * 1124) % 1537 = 1499
	   Pas de multiplication bit = 0
	3: Square: (1499 * 1499) % 1537 = 1444
	   Pas de multiplication bit = 0
	4: Square: (1444 * 1444) % 1537 = 964
	   Multiply: (964 * 681) % 1537 = 185

	=> 681 ^ 17 mod 1537 = 185


**2. Vérifier que p = 17 est un facteur de n. En déduire la valeur de ϕ(n).**

In [8]:
my_rsa.check_factor_p(p=29)
my_rsa.found_phi_n()


29 est un facteur de 1537 car n % p == 0
q = n / p = 53

ϕ(n) = (p - 1) * (q - 1)
Calcul de ϕ(n) : (p-1) * (q-1) = (29-1) * (53-1) = 1456
ϕ(n) = 1456


**3. Calculer la valeur de l’exposant de déchiffrement d. On pourra procéder par recherche exhaustive ou par recherche d’une identité de Bezout à l’aide de l’algorithme d’Euclide étendu.**

In [9]:
my_rsa.found_d()


Début du calcul de d (appelé "a" par la suite) tel que d * e ≡ 1 (mod ϕ(n)) :

Euclide étendu: a * 17 = 1 mod 1456

PGCD(17, 1456):
	17 = 0 * 1456 + 17
	1456 = 85 * 17 + 11
	17 = 1 * 11 + 6
	11 = 1 * 6 + 5
	6 = 1 * 5 + 1
	5 = 5 * 1 + 0

	=> PGCD = 1

Remontée avec Euclide étendu:
	On part de 1 = 6 - 1 * 5
	1 = 257 * 17 + -171 * 1456

=> 1 = 257 * 17 + -171 * 1456 (mod 1456) donc a = 257


**4. Calculer le message clair m2 correspondant au chiffrée c2 = 335.**

In [10]:
my_rsa = RSA(n=1537, e=17, d=257, p=29, q=53)
c = my_rsa.decrypt(335)
c = my_rsa.decrypt(185) # Pour vérifier


m = c^d mod n

Square and Multiply: 335 ^ 257 mod 1537
	bin(257) = 100000001
	0: Square: (1 * 1) % 1537 = 1
	   Multiply: (1 * 335) % 1537 = 335
	1: Square: (335 * 335) % 1537 = 24
	   Pas de multiplication bit = 0
	2: Square: (24 * 24) % 1537 = 576
	   Pas de multiplication bit = 0
	3: Square: (576 * 576) % 1537 = 1321
	   Pas de multiplication bit = 0
	4: Square: (1321 * 1321) % 1537 = 546
	   Pas de multiplication bit = 0
	5: Square: (546 * 546) % 1537 = 1475
	   Pas de multiplication bit = 0
	6: Square: (1475 * 1475) % 1537 = 770
	   Pas de multiplication bit = 0
	7: Square: (770 * 770) % 1537 = 1155
	   Pas de multiplication bit = 0
	8: Square: (1155 * 1155) % 1537 = 1446
	   Multiply: (1446 * 335) % 1537 = 255

	=> 335 ^ 257 mod 1537 = 255

m = c^d mod n

Square and Multiply: 185 ^ 257 mod 1537
	bin(257) = 100000001
	0: Square: (1 * 1) % 1537 = 1
	   Multiply: (1 * 185) % 1537 = 185
	1: Square: (185 * 185) % 1537 = 411
	   Pas de multiplication bit = 0
	2: Square: (411 * 411) % 

## **Exercice 4**

On se place dans le groupe multiplicatif (Z/53Z)∗ = {1, 2, . . . , 52} des entiers non nuls modulo 53.  

**1. On admet que 3 est un générateur. Que cela signifie-t-il ?**

In [11]:
compute = BASIC_COMPUTE()
r = compute.is_generator(53-1, 3)


3 est un générateur de l'ensemble multiplicatif modulo 53 ?
Le groupe a un ordre de 52. Nous devons vérifier si les puissances de 3 couvrent tous les entiers de 1 à 52 modulo 53
	1: 3^1 mod 53 = 3
	2: 3^2 mod 53 = 9
	3: 3^3 mod 53 = 27
	4: 3^4 mod 53 = 28
	5: 3^5 mod 53 = 31
	6: 3^6 mod 53 = 40
	7: 3^7 mod 53 = 14
	8: 3^8 mod 53 = 42
	9: 3^9 mod 53 = 20
	10: 3^10 mod 53 = 7
	11: 3^11 mod 53 = 21
	12: 3^12 mod 53 = 10
	13: 3^13 mod 53 = 30
	14: 3^14 mod 53 = 37
	15: 3^15 mod 53 = 5
	16: 3^16 mod 53 = 15
	17: 3^17 mod 53 = 45
	18: 3^18 mod 53 = 29
	19: 3^19 mod 53 = 34
	20: 3^20 mod 53 = 49
	21: 3^21 mod 53 = 41
	22: 3^22 mod 53 = 17
	23: 3^23 mod 53 = 51
	24: 3^24 mod 53 = 47
	25: 3^25 mod 53 = 35
	26: 3^26 mod 53 = 52
	27: 3^27 mod 53 = 50
	28: 3^28 mod 53 = 44
	29: 3^29 mod 53 = 26
	30: 3^30 mod 53 = 25
	31: 3^31 mod 53 = 22
	32: 3^32 mod 53 = 13
	33: 3^33 mod 53 = 39
	34: 3^34 mod 53 = 11
	35: 3^35 mod 53 = 33
	36: 3^36 mod 53 = 46
	37: 3^37 mod 53 = 32
	38: 3^38 mod 53 = 43
	39: 3^

**2. On considère un chiffrement Elgamal avec comme clé publique (101, 7, 44). Calculer le chiffré du message 89 avec comme variable auxiliaire aléatoire 15.**

In [12]:
my_elgamal = ELGAMAL(p=53, g=3, h=30)
c1, c2 = my_elgamal.encrypt(m=40, k=15)


Modular Exponentiation: 3 ^ 15 mod 53
	1: (1 * 3) % 53 = 3
	2: (3 * 3) % 53 = 9
	3: (9 * 3) % 53 = 27
	4: (27 * 3) % 53 = 28
	5: (28 * 3) % 53 = 31
	6: (31 * 3) % 53 = 40
	7: (40 * 3) % 53 = 14
	8: (14 * 3) % 53 = 42
	9: (42 * 3) % 53 = 20
	10: (20 * 3) % 53 = 7
	11: (7 * 3) % 53 = 21
	12: (21 * 3) % 53 = 10
	13: (10 * 3) % 53 = 30
	14: (30 * 3) % 53 = 37
	15: (37 * 3) % 53 = 5

	=> 3 ^ 15 mod 53 = 5

Modular Exponentiation: 30 ^ 15 mod 53
	1: (1 * 30) % 53 = 30
	2: (30 * 30) % 53 = 52
	3: (52 * 30) % 53 = 23
	4: (23 * 30) % 53 = 1
	5: (1 * 30) % 53 = 30
	6: (30 * 30) % 53 = 52
	7: (52 * 30) % 53 = 23
	8: (23 * 30) % 53 = 1
	9: (1 * 30) % 53 = 30
	10: (30 * 30) % 53 = 52
	11: (52 * 30) % 53 = 23
	12: (23 * 30) % 53 = 1
	13: (1 * 30) % 53 = 30
	14: (30 * 30) % 53 = 52
	15: (52 * 30) % 53 = 23

	=> 30 ^ 15 mod 53 = 23

Chiffrement : c1 = 3^15 mod 53
		= 5
Chiffrement : c2 = 40 * 30^15 mod 53
		= 19


**3. Vérifier que 35 est la clé privée correspondant à la clé publique ci-dessus.**

In [13]:
_ = my_elgamal.verify_private_key(x=13)


13 est la clé privée car 3^13 mod 53 = 30


**4. Déchiffrer le texte chiffré obtenu à la question 2 et retrouver le texte clair**

In [14]:
m = my_elgamal.decrypt(c1, c2)


Modular Exponentiation: 5 ^ 13 mod 53
	1: (1 * 5) % 53 = 5
	2: (5 * 5) % 53 = 25
	3: (25 * 5) % 53 = 19
	4: (19 * 5) % 53 = 42
	5: (42 * 5) % 53 = 51
	6: (51 * 5) % 53 = 43
	7: (43 * 5) % 53 = 3
	8: (3 * 5) % 53 = 15
	9: (15 * 5) % 53 = 22
	10: (22 * 5) % 53 = 4
	11: (4 * 5) % 53 = 20
	12: (20 * 5) % 53 = 47
	13: (47 * 5) % 53 = 23

	=> 5 ^ 13 mod 53 = 23

Euclide étendu: a * 23 = 1 mod 53

PGCD(23, 53):
	23 = 0 * 53 + 23
	53 = 2 * 23 + 7
	23 = 3 * 7 + 2
	7 = 3 * 2 + 1
	2 = 2 * 1 + 0

	=> PGCD = 1

Remontée avec Euclide étendu:
	On part de 1 = 7 - 3 * 2
	1 = -23 * 23 + 7 * 53

=> 1 = -23 * 23 + 7 * 53 (mod 53) donc a = 30
Déchiffrement de (5,19): m = 19 * 30 mod 53
		= 40


## **Exercice 5**

On considère la courbe elliptique définie sur le corps F7 = Z/7Z par l’équation y^2 = x^3 + 5x + 5

**1. Vérifier que les points P1 = (2, 3) et P2 = (5, 1) sont sur la courbe.**

In [20]:
my_ecc = ECC(a=5, b=5, GF=7) # curve => y^2 = x^3 + a*x + b
my_ecc.compute_points()


Calcul des points de la courbe E : y^2 = x^3 + 5*x + 5 sur le corps fini GF(7):

	Nous devons tester 49 combinaisons de points (x, y):
		Vérification : (0,0) sur E ? 0^2 = 0^3 + 5*0 + 5 (mod 7)
		=> Faux, ce point n'appartient pas à la courbe
		Vérification : (0,1) sur E ? 1^2 = 0^3 + 5*0 + 5 (mod 7)
		=> Faux, ce point n'appartient pas à la courbe
		Vérification : (0,2) sur E ? 2^2 = 0^3 + 5*0 + 5 (mod 7)
		=> Faux, ce point n'appartient pas à la courbe
		Vérification : (0,3) sur E ? 3^2 = 0^3 + 5*0 + 5 (mod 7)
		=> Faux, ce point n'appartient pas à la courbe
		Vérification : (0,4) sur E ? 4^2 = 0^3 + 5*0 + 5 (mod 7)
		=> Faux, ce point n'appartient pas à la courbe
		Vérification : (0,5) sur E ? 5^2 = 0^3 + 5*0 + 5 (mod 7)
		=> Faux, ce point n'appartient pas à la courbe
		Vérification : (0,6) sur E ? 6^2 = 0^3 + 5*0 + 5 (mod 7)
		=> Faux, ce point n'appartient pas à la courbe
		Vérification : (1,0) sur E ? 0^2 = 1^3 + 5*1 + 5 (mod 7)
		=> Faux, ce point n'appartient pas à la courbe


**2. Calculer P3 = P1 + P2.**

In [21]:
_ = my_ecc.add_two_point(P=(2, 3), Q=(5, 1))


Addition de deux points P(2, 3) et Q(5, 1):

	P + Q = (xr, yr)
		# xr = [λ^2 - xP - xQ] mod GF
		# yr = [λ * (xP - xr) - yp] mod GF
		# λ =  [(yQ - yP) / (xQ - xP)] mod GF

	=> λ = [(1 - 3) / (5 - 2)] mod 7 = 4
	=> xr = [(4^2 - 2 - 5)] mod 7 = 2
	=> yr = [4 * (2 - 2) - 3] mod 7 = 4

=> P + Q = (2, 4)


**3. Donner un encadrement du nombre de points de la courbe en utilisant la borne de Weil.**

In [17]:
my_ecc.hasse_weil_borne()


Borne de Hasse-Weil : [GF + 1 - 2 * sqrt(GF), GF + 1 + 2 * sqrt(GF)]
	=> [11 + 1 - 2 * sqrt(11), 11 + 1 + 2 * sqrt(11)]
	=> [6, 18]


**4. Calculer la liste des carrés des éléments de F7.**

In [22]:
compute = BASIC_COMPUTE()
_ = compute.list_of_squares(7)


Liste des carrés dans le corps F_7:
	0^2 mod 7 = 0
	1^2 mod 7 = 1
	2^2 mod 7 = 4
	3^2 mod 7 = 2
	4^2 mod 7 = 2
	5^2 mod 7 = 4
	6^2 mod 7 = 1

	=> Carrés distincts dans F_7 = [0, 1, 2, 4]


**5. Générer la liste de point**

In [23]:
my_ecc.generate_curve_points(P=(2,3))


Addition de deux points P(2, 3) et Q(2, 3):
	=> Les points sont égaux, doublement requis

Doublement du point P(2, 3):

	2 * P = (xr, yr)
		# xr = [λ^2 - 2 * xP] mod GF
		# yr = [λ * (xP - xr) - yp] mod GF
		# λ =  [(3 * xP^2 + a) / (2 * yP)] mod GF

	=> λ = [(3 * 2^2 + 5) / (2 * 3)] mod 7 = 4
	=> xr = [4^2 - 2 * 2] mod 7 = 5
	=> yr = [4 * (2 - 5) - 3] mod 7 = 6

=> 2 * P = (5, 6)

Addition de deux points P(5, 6) et Q(2, 3):

	P + Q = (xr, yr)
		# xr = [λ^2 - xP - xQ] mod GF
		# yr = [λ * (xP - xr) - yp] mod GF
		# λ =  [(yQ - yP) / (xQ - xP)] mod GF

	=> λ = [(3 - 6) / (2 - 5)] mod 7 = 1
	=> xr = [(1^2 - 5 - 2)] mod 7 = 1
	=> yr = [1 * (5 - 1) - 6] mod 7 = 5

=> P + Q = (1, 5)

Addition de deux points P(1, 5) et Q(2, 3):

	P + Q = (xr, yr)
		# xr = [λ^2 - xP - xQ] mod GF
		# yr = [λ * (xP - xr) - yp] mod GF
		# λ =  [(yQ - yP) / (xQ - xP)] mod GF

	=> λ = [(3 - 5) / (2 - 1)] mod 7 = 5
	=> xr = [(5^2 - 1 - 2)] mod 7 = 1
	=> yr = [5 * (1 - 1) - 5] mod 7 = 2

=> P + Q = (1, 2)

Addition

[(2, 3), (5, 6), (1, 5), (1, 2), (5, 1), (2, 4)]

**6. à 8.**  
Sur papier  

**9. Le vérifier en calculant G, [2]G, [3]G, . . . , (on s’arrête où ?).**

In [24]:
my_ecc.generate_curve_points(P=(1, 2))


Addition de deux points P(1, 2) et Q(1, 2):
	=> Les points sont égaux, doublement requis

Doublement du point P(1, 2):

	2 * P = (xr, yr)
		# xr = [λ^2 - 2 * xP] mod GF
		# yr = [λ * (xP - xr) - yp] mod GF
		# λ =  [(3 * xP^2 + a) / (2 * yP)] mod GF

	=> λ = [(3 * 1^2 + 5) / (2 * 2)] mod 7 = 2
	=> xr = [2^2 - 2 * 1] mod 7 = 2
	=> yr = [2 * (1 - 2) - 2] mod 7 = 3

=> 2 * P = (2, 3)

Addition de deux points P(2, 3) et Q(1, 2):

	P + Q = (xr, yr)
		# xr = [λ^2 - xP - xQ] mod GF
		# yr = [λ * (xP - xr) - yp] mod GF
		# λ =  [(yQ - yP) / (xQ - xP)] mod GF

	=> λ = [(2 - 3) / (1 - 2)] mod 7 = 1
	=> xr = [(1^2 - 2 - 1)] mod 7 = 5
	=> yr = [1 * (2 - 5) - 3] mod 7 = 1

=> P + Q = (5, 1)

Addition de deux points P(5, 1) et Q(1, 2):

	P + Q = (xr, yr)
		# xr = [λ^2 - xP - xQ] mod GF
		# yr = [λ * (xP - xr) - yp] mod GF
		# λ =  [(yQ - yP) / (xQ - xP)] mod GF

	=> λ = [(2 - 1) / (1 - 5)] mod 7 = 5
	=> xr = [(5^2 - 5 - 1)] mod 7 = 5
	=> yr = [5 * (5 - 5) - 1] mod 7 = 6

=> P + Q = (5, 6)

Addition

[(1, 2), (2, 3), (5, 1), (5, 6), (2, 4), (1, 5)]